In [64]:
%load_ext autoreload
%autoreload 2
import lib.mpslib.mpsfunctions as mf
import lib.mpslib.Container as CO
import lib.mpslib.TensorNetwork as TN
import lib.mpslib.MPO as MPO
import lib.mpslib.SimContainer as SCT
from lib.mpslib.Tensor import Tensor
import lib.ncon as ncon
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
tn=TN.TensorNetwork.random(shape=(3,1),tensorshapes=(2,3),minval=-0.5, maxval=0.5)
mps=TN.MPS.random(D=[10,10,10,10],d=[2,2,2])
mps.mat=np.random.random_sample(mps.mat.shape)
mps._connector=np.random.random_sample(mps.mat.shape)
mps.canonize()


AttributeError: 'numpy.ndarray' object has no attribute 'to_dense'

In [68]:
mps.position(len(mps))
print([mps.ortho_deviation(mps.get_tensor(n),'l') for n in range(len(mps))])
mps.position(0)
print([mps.ortho_deviation(mps.get_tensor(n),'r') for n in range(len(mps))])
mps.position(len(mps))
print([mps.ortho_deviation(mps.get_tensor(n),'l') for n in range(len(mps))])


[1.3772010334787358e-15, 1.7384464379002575e-15, 3.833613103607269e-12]
[3.5835356191612255e-14, 1.5813936006853799e-15, 1.0858468551596852e-15]
[1.4396455420878218e-15, 1.5830155219641351e-15, 4.043650114501581e-12]


In [29]:
mps.regauge('r',numeig=1)
print([mps.ortho_deviation(mps.get_tensor(n),'r') for n in range(len(mps))])
mps.regauge('l',numeig=1)
print([mps.ortho_deviation(mps.get_tensor(n),'l') for n in range(len(mps))])
mps.canonize(numeig=1)
print([mps.ortho_deviation(mps.get_tensor(n),'l') for n in range(len(mps))])
mps.position(0)
print([mps.ortho_deviation(mps.get_tensor(n),'r') for n in range(len(mps))])


[8.509105676757022e-14, 1.173351890955455e-15, 7.394213096205048e-16]
[1.1101212678205167e-15, 1.3956138718675127e-15, 1.2193922542498768e-13]
[1.5130983772455272e-15, 1.077612310171961e-15, 4.483017902636413e-15]
[1.0551069415866527e-13, 1.4856515470520833e-15, 1.8555463809268747e-15]


In [31]:
print(mps.TMeigs('l',ncv=20,numeig=1)[0])
print(mps.TMeigs('r',ncv=20,numeig=1)[0])

0.9999999999999247
0.9999999999998992


In [38]:
mps=TN.FiniteMPS.random(D=[1,2,3,2,1],d=[2,2,2,2])
canmps=mps.canonize('hello')
for g in canmps:
    print(mf.ortho_deviation(g,'l'))
print(canmps.D)

In [41]:
Sz=np.diag([-0.5,0.5])
canmps.measure_1site_ops([Sz for n in range(len(mps))],range(len(mps)))

[Tensor(-0.37372897),
 Tensor(-0.28734752),
 Tensor(0.23851673),
 Tensor(-0.25525447)]

In [245]:
canmps.Gamma[2].shape
print(canmps.D)
type(np.sin(canmps))
mps=canmps.toMPS()
print(type(mps))
canmps.canonize()
canmps.ortho(list(range(len(canmps))),'l')
print(type(canmps))

[1, 2, 4, 2, 1]
<class 'lib.mpslib.TensorNetwork.FiniteMPS'>
<class 'lib.mpslib.TensorNetwork.CanonizedFiniteMPS'>


In [46]:
N=100
D=10

mps=TN.FiniteMPS.random(D=[1]+[D]*(N-1)+[1],d=[2]*N,dtype=np.complex128,numpy_func=np.random.randn)
#mpo=MPO.TFI(Jx=np.ones(len(mps)),Bz=np.ones(len(mps)))
mpo=MPO.XXZ(Jz=np.ones(len(mps)-1),Jxy=np.ones(len(mps)-1),Bz=np.zeros(len(mps)))

# sim=SCT.MPSSimulation(mps,mpo)
dmrg=SCT.FiniteDMRGengine(mps,mpo)
dmrg.run_one_site(verbose=1,Econv=1E-10,Nsweeps=2,ncv=100)

SS-DMRG it=2/2, site=1/100: optimized E=-44.1139649390081416+0.0000000000000000 at D=400
reached maximum iteration number  2


array([-44.11396494])

In [59]:
canmps=mps.canonize()

In [60]:
Sz=np.diag([-0.5,0.5])
sz_1=canmps.measure_1site_ops([Sz for n in range(len(mps))],range(len(mps)))

In [61]:
Sz=np.diag([-0.5,0.5])
sz_2=mps.measure_1site_ops([Sz for n in range(len(mps))],range(len(mps)))

In [62]:
sz_1-sz_2

array([-2.77555756e-17-2.18398524e-17j,  1.21430643e-17+7.76320180e-19j,
        0.00000000e+00-9.55086529e-18j,  9.54097912e-17+5.96128474e-19j,
       -2.42861287e-17-2.60902599e-17j,  3.46944695e-17-7.03375072e-17j,
       -1.14491749e-16+2.65564554e-17j, -7.28583860e-17+3.50292981e-17j,
        8.32667268e-17+6.56051118e-19j, -1.38777878e-17-1.82209820e-17j,
       -2.42861287e-17+2.04101140e-17j,  0.00000000e+00-1.45278641e-17j,
        4.51028104e-17-1.01124134e-18j, -6.24500451e-17-1.32792825e-17j,
        6.59194921e-17-3.40607559e-17j, -7.63278329e-17-2.05060573e-17j,
        3.81639165e-17+3.34046026e-17j, -6.59194921e-17+1.07193160e-17j,
        6.59194921e-17+2.64371255e-17j, -1.07552856e-16-1.06093483e-17j,
        1.38777878e-17+1.65188572e-17j,  5.89805982e-17-3.32008843e-17j,
        3.12250226e-17+3.71623539e-17j, -4.51028104e-17-3.71558525e-17j,
        1.66533454e-16+6.17273331e-17j, -5.55111512e-17-1.22354709e-18j,
        1.45716772e-16+9.80935547e-18j, -1.76941795

In [26]:
dmrg.run_one_site(verbose=1,Econv=1E-10,Nsweeps=10,cp=1)

SS-DMRG it=1/10, site=1/100: optimized E=-44.1146468899851811+0.0000000000000000 at D=400

PicklingError: Can't pickle <class 'lib.mpslib.TensorNetwork.FiniteMPS'>: it's not the same object as lib.mpslib.TensorNetwork.FiniteMPS

In [7]:
print(dmrg.mps.D)
!ls

[1, 2, 4, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 4, 2, 1]
2019123_TFI_VUMPSD128_Jx-1.0_B1.0
2019123_TFI_VUMPSD256_Jx-1.0_B1.0
201918_TFI_ITEBDD32_Jx-1.0_B1.0_dt(-0-0.01j)
201918_TFI_ITEBDD8_Jx-1.0_B1.0_dt(-0-0.01j)
201918_TFI_VUMPSD32_Jx-1.0_B1.0
201918_TFI_VUMPS_TDVPD32_Jx-1.0_B1.0_dt(-0-0.01j)
201918_TFI_VUMPS_TDVPD32_Jx-1.0_B1.0_dt(0.1+0j)
201918_TFI_VUMPS_TDVPD32_Jx-1.0_B1.0_dt(1+0j)
201918_TFI_VUMPS_TDVPD32_Jx-1.0_B1.0_dt0.1j
201918_TFI_VUMPS_TDVPD32_Jx-1.0_B1.0_dt1j
Branching.py
DMRG_dmrg_cp.pickle
HeisIDMRG.py
HeisIMPS.py
HeisMPS.py
HeisVUMPS.py
ITEBD.py
ITEBD.py~
TDVPproductState.py
TEBD.py
TEBDproductState.py
TFI.py
TFIIDMRG.py
TFITDVP.py
TFI_V

In [10]:
dmrg2=SCT.DMRGengine.read('DMRG_dmrg_cp.pickle')

In [11]:
print(dmrg2.mps.D)

[1, 2, 4, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 8, 4, 2, 1]


1
2


array([-44.11635951])